In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [9]:
import src.preprocessing as pre
import numpy as np

In [4]:
meta, y, mapping_dict = pre.load_data('../data/meta.json','../data/y.csv','../data/mappings.json')

In [13]:
a = pre.decode(meta,mapping_dict).Ranch.unique()






In [12]:
b = np.array([
    'VAP', 'OAP', 'OIB', 'OLB', 'VDG', 'SMB', 'OSF', 'SGB', 
    'SEB', 'TJC', 'ZCC', 'ZJL', 'ZJM', 'ZJZ', 'ZPR', 'ZVJ'
])

In [18]:
np.unique(np.concatenate([a,b]))

array(['MAP', 'OAP', 'OIB', 'OLB', 'OSF', 'SAB', 'SEB', 'SFB', 'SGB',
       'SGC', 'SJB', 'SLB', 'SMB', 'SMR', 'TJC', 'VAP', 'VDG', 'ZCC',
       'ZJL', 'ZJM', 'ZJZ', 'ZPR', 'ZVJ'], dtype=object)

In [20]:
grower_coordinates = {
    'MAP': (31.666431, -116.515308),
    'OAP': (31.920693, -116.243753),
    'OIB': (31.917818, -116.241634),  # estimated
    'OLB': (31.907724, -116.236947),
    'OSF': (31.925036, -116.244201),
    'SAB': (32.026187, -116.742880),
    'SEB': (32.017032, -116.689125),  # estimated
    'SFB': (32.025498, -116.753590),
    'SGB': (32.016094, -116.763115),
    'SGC': (32.016094, -116.763115),  # estimated
    'SJB': (32.022490, -116.754103),
    'SLB': (32.024978, -116.739198),
    'SMB': (31.907241, -116.241314),
    'SMR': (32.096734, -116.829672),
    'VAP': (31.421453, -116.269149),
    'VDG': (31.345217, -116.326015),
    'TJC': (25.091810, -111.315840),
    'ZCC': (27.335660, -113.302957),  # estimated
    'ZJL': (28.301185, -114.028883),  # estimated - same as ZJM
    'ZJM': (28.301185, -114.028883),
    'ZJZ': (28.301185, -114.028883),  # estimated - same as ZJM
    'ZPR': (25.419456, -111.885169),
    'ZVJ': (28.286338, -113.994819)
}